# DATGAN JUPYTER NOTEBOOK

##### This notebook can be used to generate synthtic census dataset at four spatial level: Nomenclature of Territorial Units for Statistics(NUTS 2), Distrito, Município and Freguesia. This is an interactive notebook and has been developed as part of the Master's in Data Science dissertation.

In [1]:
!pip install --upgrade pip
!pip install tensorflow
!pip install FuzzyTM
!pip install Cython
!pip install blosc2==2.0.0
!pip install protobuf==3.20.0 --user
!pip install datgan --user
!pip install shapely
!pip install geopandas

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [2]:
# IMPOTING LIBRARIES
import datgan
from datgan import DATGAN, advise

import numpy as np
import pandas as pd
import networkx as nx
import tensorflow as tf

import random
import math
import json
import os
import datetime
import string
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, MultiPoint
import geopandas as gpd
from scipy.spatial import Voronoi
from sklearn.cluster import KMeans

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
df_full = pd.read_csv(r"/root/Portugal_Original.csv", index_col=0)

df_full.columns.tolist()

i = 1
rename_cols = {}
for j in df_full.columns.tolist():
    rename_cols[j] = f"{i:0{5}d}"
    i = i + 1
rename_cols

df_full.rename(columns = rename_cols, inplace=True)
df_full.info()

df_corr = df_full.corr()

# Function to find highly correlated variables for a given variable
def find_highly_correlated_values(df_full, variable, threshold=0.1):
    correlated_vals = df_corr[variable]
    highly_correlated_vals = correlated_vals[correlated_vals > threshold].drop(variable)
    return highly_correlated_vals

# Loop through each variable and find highly correlated variables
graph_nodes_list = []
temp_col = []
no_corr_vars = []
for column in df_full.columns:
    highly_correlated_vals = find_highly_correlated_values(df_full, column)
    temp_col.append(column)
    if len(highly_correlated_vals) == 0:
        print('No correlation found for variable: ' + column)
        print('Add variable ' + column + ' as single node to the graph')
        no_corr_vars.append(column)
    else:
        for i in highly_correlated_vals.keys():
            if i not in temp_col:
                graph_nodes_temp = (column, i)
                graph_nodes_list.append(graph_nodes_temp)
                
graph = nx.DiGraph()
graph.add_edges_from(graph_nodes_list)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210170 entries, 0 to 210169
Data columns (total 27 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   00001   210170 non-null  int64
 1   00002   210170 non-null  int64
 2   00003   210170 non-null  int64
 3   00004   210170 non-null  int64
 4   00005   210170 non-null  int64
 5   00006   210170 non-null  int64
 6   00007   210170 non-null  int64
 7   00008   210170 non-null  int64
 8   00009   210170 non-null  int64
 9   00010   210170 non-null  int64
 10  00011   210170 non-null  int64
 11  00012   210170 non-null  int64
 12  00013   210170 non-null  int64
 13  00014   210170 non-null  int64
 14  00015   210170 non-null  int64
 15  00016   210170 non-null  int64
 16  00017   210170 non-null  int64
 17  00018   210170 non-null  int64
 18  00019   210170 non-null  int64
 19  00020   210170 non-null  int64
 20  00021   210170 non-null  int64
 21  00022   210170 non-null  int64
 22  00023   210170 non-n

In [5]:
# COLLECTING USER INPUTS

while True:
    try:
        temp_gate = input("Would you like to specify the number of units in each spatial level?\nIf yes, enter 1; otherwise enter 0: ")
        temp_gate = int(temp_gate)
        if temp_gate not in [0,1]:
            print("\nInvalid input. Please enter a valid number.")
            continue
        break
    except ValueError:
        print("\nInvalid input. Please enter an integer.")
        
if temp_gate == 1:
    while True:
        try:
            num_levels = input("Please enter the number of spatial levels: ")
            num_levels = int(num_levels)
            count_per_level =[]
            for i in range(1, num_levels+1):
                sent = f'Please enter the number of units in level {i}: '
                unit_count = input(sent)
                unit_count = int(unit_count)
                count_per_level.append(unit_count)
            flag = 0
            for j in range(len(count_per_level)-1):
                if count_per_level[j] < count_per_level[j + 1] or 0 in count_per_level:
                    print("\nInvalid input. Please re-enter values.\nTip: \n1. Do not enter zeroes. \n2. Enter units in descending order per level, with the lowest level having the most units.")
                    flag = 1
                    break
            if flag != 0:
                continue
            else:
                break
        except ValueError:
            print("\nInvalid input. Please enter an integer.")

Would you like to specify the number of units in each spatial level?
If yes, enter 1; otherwise enter 0:  1
Please enter the number of spatial levels:  5
Please enter the number of units in level 1:  221000
Please enter the number of units in level 2:  215
Please enter the number of units in level 3:  92
Please enter the number of units in level 4:  26
Please enter the number of units in level 5:  7


In [6]:
# CREATING THE RESULT DIRECTORY

current_datetime = datetime.datetime.now().strftime('%d%m%Y_%H%M%S')
result_folder_name = f'Portugal_synthetic_census_dataset_{current_datetime}'
result_folder_path = os.path.join(os.getcwd(), 'results', result_folder_name)
os.makedirs(result_folder_path, exist_ok=True)

In [7]:
# LOADING THE MODEL

df = pd.read_csv(r'/root/Portugal_Original.csv', index_col=0)
i = 1
rename_cols = {}
for j in df.columns.tolist():
    rename_cols[j] = f"{i:0{5}d}"
    i = i + 1
rename_cols
df.rename(columns = rename_cols, inplace=True)

#with open(os.path.join(os.getcwd(), 'datgan_model\dag', 'dag.json'), 'r') as file:
    #dag_list = json.load(file)
    #dag_nodes_list = []
    #for i in dag_list:
        #dag_nodes_list.append(tuple(i))
        
#graph = nx.DiGraph()
#graph.add_edges_from(dag_nodes_list)

datgan = DATGAN(output=r"test/model/", loss_function='WGGP', label_smoothing='TS', learning_rate=0.0001, batch_size=500, num_epochs=100)

datgan.load(df, dag=graph, preprocessed_data_path=r"Portugal25/encoded_data")

2024-09-21 19:59:15.748110: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-21 19:59:16.865485: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22167 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090 D, pci bus id: 0000:17:00.0, compute capability: 8.9


Preprocessed data have been loaded!


In [8]:



# GENERATING TABULAR SYNTHETIC CENSUS DATA

if temp_gate == 1 and len(count_per_level) > 0:
    num_samples = count_per_level[0]
else:
    num_samples = random.randint(210100, 2101000)

samples = datgan.sample(num_samples, sampling='SS')

samples.to_csv(r"results/Portugal_synthetic_census_dataset.csv", index=False)

Sampling from DATGAN:   0%|          | 0/221000 [00:00<?, ?it/s]2024-09-21 19:59:35.703719: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2024-09-21 19:59:37.059753: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Sampling from DATGAN: 100%|██████████| 221000/221000 [01:32<00:00, 2385.03it/s]


In [10]:
# GENERATING SYNTHETIC SPATIAL BOUNDARIES AT LOWEST LEVEL

x_min = np.random.uniform(70000, 90000)
x_max = np.random.uniform(400000, 800000)
y_min = np.random.uniform(6000, 20000)
y_max = np.random.uniform(400000, 800000)

num_coords = random.randint(5, 15)

x = np.random.uniform(x_min, x_max, num_coords)
y = np.random.uniform(y_min, y_max, num_coords)

poly_points = np.column_stack((x, y))

# Calculate the centroid of the points
centroid = np.mean(poly_points, axis=0)

# Calculate the polar angles of the points with respect to the centroid
angles = np.arctan2(poly_points[:, 1] - centroid[1], poly_points[:, 0] - centroid[0])

# Sort the points based on the angles in clockwise direction
sorted_indices = np.argsort(angles)
poly_points = poly_points[sorted_indices]
poly_points = np.array(poly_points)

# Create a Polygon object
boundary_polygon = Polygon(poly_points)
exterior_coords = boundary_polygon.exterior.coords
x_coords, y_coords = zip(*exterior_coords)

points = []
while len(points) < num_samples:
    point = (random.uniform(min(x_coords), max(x_coords)),
             random.uniform(min(y_coords), max(y_coords)))
    if boundary_polygon.contains(Point(point)):
        points.append(point)
points = np.array(points)

def voronoi_finite_polygons_2d(vor, radius=None):
    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue
        
        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            t = vor.points[p2] - vor.points[p1] # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)

vor = Voronoi(points)

regions, vertices = voronoi_finite_polygons_2d(vor)

clipped_polygons = []
for region in regions:
    clipped_polygon = Polygon(vertices[region])
    clipped_polygon = clipped_polygon.intersection(boundary_polygon)
    clipped_polygons.append(clipped_polygon)
    
census_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(clipped_polygons), crs="EPSG:4326")
census_gdf = pd.concat([census_gdf, samples], axis=1)

In [11]:
# GENERATING SYNTHETIC SPATIAL BOUNDARIES AT OTHER LEVELS

if temp_gate == 1 and len(count_per_level) == 1:
    start_lts = random.sample(string.ascii_uppercase, 2)
    code_start = ''.join(start_lts)
    code_list = [f"{code_start}{i:0{len(str(num_samples))}d}" for i in range(1, num_samples+1)]
    census_gdf.insert(0, 'Code', code_list)
    census_gdf.to_file(os.path.join(result_folder_path, 'synthetic_census_dataset_level_1.shp'))
else:
    level_count_dict = {}
    
    if temp_gate == 1 and len(count_per_level) > 1:
        for val in count_per_level[1:]:
            idx = count_per_level.index(val) + 1
            level_count_dict[idx] = val
    #else:
        #level_count_dict[2] = random.randint(35000, 40000)
        #level_count_dict[3] = random.randint(6000, 10000)
        #level_count_dict[4] = random.randint(300, 600)
        #level_count_dict[5] = random.randint(8, 14)
    
    for lvl, clusters in level_count_dict.items():
        census_centroids = census_gdf.geometry.apply(lambda geom: [geom.centroid.x, geom.centroid.y]).tolist()
        kmeans = KMeans(n_clusters=clusters, init='k-means++', n_init='auto')
        current_spatial_lvl = f'level_{lvl}'
        census_gdf[current_spatial_lvl] = kmeans.fit_predict(census_centroids)
        start_lts = random.sample(string.ascii_uppercase, 2)
        code_start = ''.join(start_lts)
        code_list = [f"{code_start}{i:0{len(str(num_samples))}d}" for i in range(1, len(census_gdf)+1)]
        census_gdf.insert(0, 'Code', code_list)
        shapefile_name = f'synthetic_census_dataset_level_{lvl-1}.shp'
        census_gdf.to_file(os.path.join(result_folder_path, shapefile_name))
        census_gdf.drop(columns= ["Code"], axis=1, inplace = True)
        census_gdf = census_gdf.dissolve(by=current_spatial_lvl, aggfunc='sum')
        census_gdf.reset_index(drop = True, inplace = True)
        
    start_lts = random.sample(string.ascii_uppercase, 2)
    code_start = ''.join(start_lts)
    code_list = [f"{code_start}{i:0{len(str(num_samples))}d}" for i in range(1, len(census_gdf)+1)]
    census_gdf.insert(0, 'Code', code_list)
    final_shapefile_name = f'synthetic_census_dataset_level_{lvl}.shp'
    census_gdf.to_file(os.path.join(result_folder_path, final_shapefile_name))
    
print('Process successfully completed!')

Process successfully completed!
